In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.CodeMirror {font-family:Consolas; font-size:10pt;}
div.output {font-size:10pt;}
div.input {font-family:Consolas; font-size:10pt; font-weight:bold;}
div.prompt {min-width:70px;}
</style>
"""))

**<font size='6' color='darkred'>ch04. RNN</font>**

# 문맥을 이용하여 모델만들기

In [2]:
text = """경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다
"""

In [12]:
from keras_preprocessing.text import Tokenizer

t = Tokenizer()
t.fit_on_texts([text])
encoded = t.texts_to_sequences([text])

print(encoded)
print(t.word_index)

[[2, 3, 1, 4, 5, 6, 1, 7, 8, 1, 9, 10, 1, 11]]
{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [15]:
t.texts_to_sequences(['경마장에 있는 말이 뛰고 있다'])[0]

[2, 3, 1, 4, 5]

In [16]:
sequences = []

for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    
    print('원래 문장: ', line)
    print('encoded 문장: ', encoded)
    
    for i in range(0, len(encoded)-1):
        for j in range(i+2, len(encoded)+1):
            sequences.append(encoded[i:j])
print()

for sequence in sequences:
    print('[', end='')
    for word_seq in sequence:
        for key, value in t.word_index.items():
            if value == word_seq:
                print("{}:{}".format(value, key), end='') 
    print(']')

원래 문장:  경마장에 있는 말이 뛰고 있다
encoded 문장:  [2, 3, 1, 4, 5]
원래 문장:  그의 말이 법이다
encoded 문장:  [6, 1, 7]
원래 문장:  가는 말이 고와야 오는 말이 곱다
encoded 문장:  [8, 1, 9, 10, 1, 11]
원래 문장:  
encoded 문장:  []

[2:경마장에3:있는]
[2:경마장에3:있는1:말이]
[2:경마장에3:있는1:말이4:뛰고]
[2:경마장에3:있는1:말이4:뛰고5:있다]
[3:있는1:말이]
[3:있는1:말이4:뛰고]
[3:있는1:말이4:뛰고5:있다]
[1:말이4:뛰고]
[1:말이4:뛰고5:있다]
[4:뛰고5:있다]
[6:그의1:말이]
[6:그의1:말이7:법이다]
[1:말이7:법이다]
[8:가는1:말이]
[8:가는1:말이9:고와야]
[8:가는1:말이9:고와야10:오는]
[8:가는1:말이9:고와야10:오는1:말이]
[8:가는1:말이9:고와야10:오는1:말이11:곱다]
[1:말이9:고와야]
[1:말이9:고와야10:오는]
[1:말이9:고와야10:오는1:말이]
[1:말이9:고와야10:오는1:말이11:곱다]
[9:고와야10:오는]
[9:고와야10:오는1:말이]
[9:고와야10:오는1:말이11:곱다]
[10:오는1:말이]
[10:오는1:말이11:곱다]
[1:말이11:곱다]


In [17]:
sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [3, 1],
 [3, 1, 4],
 [3, 1, 4, 5],
 [1, 4],
 [1, 4, 5],
 [4, 5],
 [6, 1],
 [6, 1, 7],
 [1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11],
 [1, 9],
 [1, 9, 10],
 [1, 9, 10, 1],
 [1, 9, 10, 1, 11],
 [9, 10],
 [9, 10, 1],
 [9, 10, 1, 11],
 [10, 1],
 [10, 1, 11],
 [1, 11]]

In [18]:
print([len(s) for s in sequences]) 

[2, 3, 4, 5, 2, 3, 4, 2, 3, 2, 2, 3, 2, 2, 3, 4, 5, 6, 2, 3, 4, 5, 2, 3, 4, 2, 3, 2]


In [19]:
# sequences에 제일 많은 단어가 들어있는 부분의 갯수 
maxlen = max([len(s) for s in sequences])
maxlen

6

In [20]:
type(sequences), len(sequences)

(list, 28)

In [21]:
# sequences를 훈련가능한 데이터로 만들기
from tensorflow.keras.preprocessing.sequence import pad_sequences

sequences = pad_sequences(sequences=sequences,
                          maxlen=maxlen,
                          padding='pre')
type(sequences), len(sequences), sequences

(numpy.ndarray,
 28,
 array([[ 0,  0,  0,  0,  2,  3],
        [ 0,  0,  0,  2,  3,  1],
        [ 0,  0,  2,  3,  1,  4],
        [ 0,  2,  3,  1,  4,  5],
        [ 0,  0,  0,  0,  3,  1],
        [ 0,  0,  0,  3,  1,  4],
        [ 0,  0,  3,  1,  4,  5],
        [ 0,  0,  0,  0,  1,  4],
        [ 0,  0,  0,  1,  4,  5],
        [ 0,  0,  0,  0,  4,  5],
        [ 0,  0,  0,  0,  6,  1],
        [ 0,  0,  0,  6,  1,  7],
        [ 0,  0,  0,  0,  1,  7],
        [ 0,  0,  0,  0,  8,  1],
        [ 0,  0,  0,  8,  1,  9],
        [ 0,  0,  8,  1,  9, 10],
        [ 0,  8,  1,  9, 10,  1],
        [ 8,  1,  9, 10,  1, 11],
        [ 0,  0,  0,  0,  1,  9],
        [ 0,  0,  0,  1,  9, 10],
        [ 0,  0,  1,  9, 10,  1],
        [ 0,  1,  9, 10,  1, 11],
        [ 0,  0,  0,  0,  9, 10],
        [ 0,  0,  0,  9, 10,  1],
        [ 0,  0,  9, 10,  1, 11],
        [ 0,  0,  0,  0, 10,  1],
        [ 0,  0,  0, 10,  1, 11],
        [ 0,  0,  0,  0,  1, 11]]))

In [22]:
# 독립변수(x)와 종속변수(y) 로 분리
x = sequences[:, :-1]
y = sequences[:, -1]
y

array([ 3,  1,  4,  5,  1,  4,  5,  4,  5,  5,  1,  7,  7,  1,  9, 10,  1,
       11,  9, 10,  1, 11, 10,  1, 11,  1, 11, 11])

In [23]:
print(t.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [29]:
voca_size = len(t.word_index)+1
voca_size

12

In [39]:
# y 원핫 인코딩
from tensorflow.keras.utils import to_categorical

y = to_categorical(y, num_classes=voca_size)
y

array([[[[0., 1., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         ...,
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.]],

        [[0., 1., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         ...,
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.]],

        [[0., 1., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         ...,
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 1., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         ...,
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
    

In [40]:
x.shape[1]

5

In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding

# RNN 모델 생성
model = Sequential()
# 희소행렬로 변환 (10:벡터)
model.add(Embedding(voca_size, 10, input_length=x.shape[1]))
model.add(SimpleRNN(32))
model.add(Dense(voca_size, activation="softmax"))

# 모델 학습과정 설정
model.compile(loss="categorical_crossentropy", 
              optimizer="adam", 
              metrics=['accuracy'])

# 학습시키기
hist = model.fit(x, y, epochs=300, verbose=0)

ValueError: in user code:

    C:\Users\mupie\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\mupie\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\mupie\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\mupie\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\mupie\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\mupie\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
 